In [ ]:
import time
!pip install intrinio-sdk
import intrinio_sdk
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from intrinio_sdk.rest import ApiException
from pprint import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.0 MB/s eta 0:00:00


In [ ]:
def getDictionaryFromXLSFile(XLSFile, ratings):
    wordlist = {}
    for i in range(1,8):
        frame = XLSFile.parse(XLSFile.sheet_names[i], header=None)
        frame.columns = ['word']
        frame['score'] = ratings[i - 1]
        dict =frame.set_index('word').to_dict()
        wordlist = {**wordlist, **dict} # append dictionary to wordlist
    return wordlist

In [ ]:
def updateLexicon(sia):
    file_path = 'LoughranMcDonald_SentimentWordLists_2018.xlsx'
    file = pd.ExcelFile(file_path)

    wordlist = getDictionaryFromXLSFile(file, [-1,1,-1,-1,1,-1,-1]) # assign scores to words from financial lexiconn


    sia.lexicon.update(wordlist)

In [ ]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
updateLexicon(sia)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def inPassage(passage, list):
    for word in list:
        if word in passage:
            return True
    return False

In [ ]:
def createColumn(df, columnName, list):
    df[columnName] = df['summary'].apply(inPassage, args=(list,))

In [34]:
# create dictionaries of words relating to each pillar of ESG. code can be improved by adding to these
eWords = {
    'environment', 'climate', 'sustainable', 'sustainability', 'green', 'planet',
    'biodiversity', 'conservation', 'eco-friendly', 'emissions', 'energy', 'environmental impact',
    'pollution', 'recycling', 'renewable resources', 'waste management', 'water conservation',
    'carbon footprint', 'deforestation', 'sustainable development', 'wildlife conservation',
    'renewable', 'solar', 'wind', 'hydro', 'air quality', 'soil pollution', 'water pollution'
}
sWords = {
    'social', 'community', 'responsibility', 'philanthropy', 'charity',
    'equity', 'inclusion', 'diversity', 'human rights', 'labor practices',
    'community engagement', 'employee well-being', 'fair trade', 'social impact',
    'social justice', 'accessibility', 'health and safety', 'education support',
    'equal opportunity', 'social welfare'
}
gWords = {
    'governance', 'board', 'administration', 'organization', 'conduct',
    'ethics', 'compliance', 'transparency', 'accountability', 'risk management',
    'stakeholder engagement', 'corporate citizenship', 'integrity', 'leadership',
    'executive compensation', 'shareholder rights', 'corporate ethics', 'audit committee',
    'regulatory compliance', 'anti-corruption'
}

In [ ]:
def getSentimentScore(passage):
    try:
        score = sia.polarity_scores(str(passage))['compound']
    except TypeError:
        score = 0
    return score

In [35]:
def getSentimentScore(passage):
    try:
        # Get the original compound sentiment score from VADER
        original_score = sia.polarity_scores(str(passage))['compound']
        # Convert the original score to the new scale
        new_score = 50 - (50 * original_score)
    except TypeError:
        new_score = 50  # Assuming neutral sentiment in case of an error
    return new_score

In [ ]:
def getESGscore(ticker):
    # Manually written news article
    news_articles = [
        {
            'summary': 'The company has made significant strides in reducing its carbon footprint, launching several new sustainable products that have been well received by the market.',
            'source': 'Manual Source',
            'date': '2024-03-10',
            # Add more fields as necessary
        },
        # You can add more articles in this list
    ]

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(news_articles)

    # Assuming eWords, sWords, gWords are defined elsewhere in your code
    createColumn(df, 'eContent', eWords)
    createColumn(df, 'sContent', sWords)
    createColumn(df, 'gContent', gWords)

    # Now you can calculate the sentiment score for each article and further compute ESG scores as before
    df['sentiment'] = df['summary'].apply(getSentimentScore)

    # The calculation of ESG scores based on sentiment scores can remain unchanged
    eScore = df[df['eContent']].sum()['sentiment'] / len(df)
    sScore = df[df['sContent']].sum()['sentiment'] / len(df)
    gScore = df[df['gContent']].sum()['sentiment'] / len(df)

    return [eScore, sScore, gScore]

# Example use of the modified function
scores = getESGscore("ManualTicker")
print("Environment: {0}, Social: {1}, Governance: {2}".format(scores[0], scores[1], scores[2]))

Environment: 27.98, Social: 0.0, Governance: 0.0


In [26]:
company=pd.read_excel("Company_list.xlsx")
company.head()

,S.No,Company,n1,n2,n3,n4,concat
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...


In [ ]:
scores

[27.98, 0.0, 0.0]

In [36]:
def getESGscore(news):
    # Manually written news article
    news_articles = [
        {
            'summary': news,

            # Add more fields as necessary
        },
        # You can add more articles in this list
    ]

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(news_articles)

    # Assuming eWords, sWords, gWords are defined elsewhere in your code
    createColumn(df, 'eContent', eWords)
    createColumn(df, 'sContent', sWords)
    createColumn(df, 'gContent', gWords)

    # Now you can calculate the sentiment score for each article and further compute ESG scores as before
    df['sentiment'] = df['summary'].apply(getSentimentScore)

    # The calculation of ESG scores based on sentiment scores can remain unchanged
    eScore = df[df['eContent']].sum()['sentiment'] / len(df)
    sScore = df[df['sContent']].sum()['sentiment'] / len(df)
    gScore = df[df['gContent']].sum()['sentiment'] / len(df)

    return [eScore, sScore, gScore]

In [37]:
getESGscore("The company has made significant strides in reducing its carbon footprint, launching several new sustainable products that have been well received by the market.")

[27.98, 0.0, 0.0]

In [ ]:
envi=[]
soci=[]
gov=[]
for i in range(36):
  score=getESGscore(company.loc[i,"concat"])
  envi.append(score[0])
  soci.append(score[1])
  gov.append(score[2])

In [ ]:
company.loc[0,"concat"]

"Cognizant has announced a major investment in green computing technologies, aiming to reduce carbon emissions from its data centers by 40% by 2025. This initiative places the company at the forefront of sustainable IT services, responding to growing client demand for environmentally friendly digital solutions. Critics argue, however, that the company's overall carbon footprint remains substantial, urging broader action across all operationsCognizant has successfully achieved its goal of filling 30% of leadership roles with women and underrepresented minorities, a significant milestone in its ESG journey. This accomplishment reflects the company's commitment to creating a diverse and inclusive workplace. Nonetheless, some employee groups call for more transparent career progression pathways and mentorship programs to ensure equitable growth opportunities for allRecent allegations have surfaced accusing Cognizant of misreporting certain ESG metrics, casting doubt on the integrity of its

In [ ]:
len(envi)

36

In [ ]:
gov

[0.0,
 0.8200000000000003,
 0.18999999999999773,
 1.8249999999999957,
 2.0899999999999963,
 0.4350000000000023,
 0.0,
 1.0949999999999989,
 1.5649999999999977,
 0.779999999999994,
 0.0,
 0.2049999999999983,
 0.9750000000000014,
 0.35999999999999943,
 14.519999999999996,
 0.5649999999999977,
 0.0,
 0.32499999999999574,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6.035000000000004,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
len(soci)

36

In [ ]:
company["eScore"]=envi
company["sScore"]=soci
company["gScore"]=gov
company.head()

,S.No,Company,n1,n2,n3,n4,concat,eScore,sScore,gScore
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...,2.225,2.225,0.000
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...,0.820,0.000,0.820
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...,0.190,0.190,0.190
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...,1.825,1.825,1.825
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...,2.090,0.000,2.090


In [ ]:
company.to_csv("esg_score.csv")

In [44]:
e1=[]
s1=[]
g1=[]
for i in range(36):
  score=getESGscore(company.loc[i,"n1"])
  e1.append(score[0])
  s1.append(score[1])
  g1.append(score[2])

company["e1"]=e1
company["s1"]=s1
company["g1"]=g1
company.head()

,S.No,Company,n1,n2,n3,n4,concat,e1,s1,g1,...,g2,e3,s3,g3,e4,s4,g4,eScore,sScore,gScore
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...,34.090,0.0,0.0,...,3.035,16.475,0.000,16.475,10.930,10.930,0.000,15.37375,2.73250,4.87750
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...,37.500,0.0,0.0,...,0.000,0.000,3.090,3.090,20.030,20.030,20.030,14.38250,7.20500,5.78000
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...,15.460,0.0,0.0,...,12.080,0.000,0.000,3.200,2.275,2.275,2.275,4.43375,3.58875,4.38875
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...,20.030,0.0,0.0,...,10.180,0.000,4.595,0.000,0.000,7.065,7.065,5.00750,5.46000,4.31125
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...,17.015,0.0,0.0,...,0.000,0.000,7.830,0.000,0.000,14.985,14.985,4.25375,5.70375,3.74625


In [45]:
e1=[]
s1=[]
g1=[]
for i in range(36):
  score=getESGscore(company.loc[i,"n2"])
  e1.append(score[0])
  s1.append(score[1])
  g1.append(score[2])

company["e2"]=e1
company["s2"]=s1
company["g2"]=g1
company.head()

,S.No,Company,n1,n2,n3,n4,concat,e1,s1,g1,...,g2,e3,s3,g3,e4,s4,g4,eScore,sScore,gScore
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...,34.090,0.0,0.0,...,3.035,16.475,0.000,16.475,10.930,10.930,0.000,15.37375,2.73250,4.87750
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...,37.500,0.0,0.0,...,0.000,0.000,3.090,3.090,20.030,20.030,20.030,14.38250,7.20500,5.78000
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...,15.460,0.0,0.0,...,12.080,0.000,0.000,3.200,2.275,2.275,2.275,4.43375,3.58875,4.38875
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...,20.030,0.0,0.0,...,10.180,0.000,4.595,0.000,0.000,7.065,7.065,5.00750,5.46000,4.31125
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...,17.015,0.0,0.0,...,0.000,0.000,7.830,0.000,0.000,14.985,14.985,4.25375,5.70375,3.74625


In [46]:
e1=[]
s1=[]
g1=[]
for i in range(36):
  score=getESGscore(company.loc[i,"n3"])
  e1.append(score[0])
  s1.append(score[1])
  g1.append(score[2])

company["e3"]=e1
company["s3"]=s1
company["g3"]=g1
company.head()

,S.No,Company,n1,n2,n3,n4,concat,e1,s1,g1,...,g2,e3,s3,g3,e4,s4,g4,eScore,sScore,gScore
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...,34.090,0.0,0.0,...,3.035,16.475,0.000,16.475,10.930,10.930,0.000,15.37375,2.73250,4.87750
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...,37.500,0.0,0.0,...,0.000,0.000,3.090,3.090,20.030,20.030,20.030,14.38250,7.20500,5.78000
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...,15.460,0.0,0.0,...,12.080,0.000,0.000,3.200,2.275,2.275,2.275,4.43375,3.58875,4.38875
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...,20.030,0.0,0.0,...,10.180,0.000,4.595,0.000,0.000,7.065,7.065,5.00750,5.46000,4.31125
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...,17.015,0.0,0.0,...,0.000,0.000,7.830,0.000,0.000,14.985,14.985,4.25375,5.70375,3.74625


In [47]:
e1=[]
s1=[]
g1=[]
for i in range(36):
  score=getESGscore(company.loc[i,"n4"])
  e1.append(score[0])
  s1.append(score[1])
  g1.append(score[2])

company["e4"]=e1
company["s4"]=s1
company["g4"]=g1
company.head()

,S.No,Company,n1,n2,n3,n4,concat,e1,s1,g1,...,g2,e3,s3,g3,e4,s4,g4,eScore,sScore,gScore
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...,34.090,0.0,0.0,...,3.035,16.475,0.000,16.475,10.930,10.930,0.000,15.37375,2.73250,4.87750
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...,37.500,0.0,0.0,...,0.000,0.000,3.090,3.090,20.030,20.030,20.030,14.38250,7.20500,5.78000
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...,15.460,0.0,0.0,...,12.080,0.000,0.000,3.200,2.275,2.275,2.275,4.43375,3.58875,4.38875
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...,20.030,0.0,0.0,...,10.180,0.000,4.595,0.000,0.000,7.065,7.065,5.00750,5.46000,4.31125
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...,17.015,0.0,0.0,...,0.000,0.000,7.830,0.000,0.000,14.985,14.985,4.25375,5.70375,3.74625


In [48]:
company["eScore"]=(company["e1"]+company["e2"]+company["e3"]+company["e4"])/4
company["sScore"]=(company["s1"]+company["s2"]+company["s3"]+company["s4"])/4
company["gScore"]=(company["g1"]+company["g2"]+company["g3"]+company["g4"])/4
company.head()

,S.No,Company,n1,n2,n3,n4,concat,e1,s1,g1,...,g2,e3,s3,g3,e4,s4,g4,eScore,sScore,gScore
0,1,COGNIZANT,Cognizant has announced a major investment in ...,Cognizant has successfully achieved its goal o...,Recent allegations have surfaced accusing Cogn...,Cognizant has launched a new suite of ESG cons...,Cognizant has announced a major investment in ...,34.090,0.0,0.0,...,3.035,16.475,0.000,16.475,10.930,10.930,0.000,15.37375,2.73250,4.87750
1,2,MICROSOFT,Microsoft has set an ambitious goal to become ...,Microsoft has been recognized for its commitme...,Recent reports have raised concerns about labo...,Microsoft's AI for Earth program has been maki...,Microsoft has set an ambitious goal to become ...,37.500,0.0,0.0,...,0.000,0.000,3.090,3.090,20.030,20.030,20.030,14.38250,7.20500,5.78000
2,3,ACCENTURE,Accenture has proudly announced that 100% of i...,Accenture has been recognized for its leadersh...,Amidst growing reliance on digital technologie...,Accenture has expanded its sustainability serv...,Accenture has proudly announced that 100% of i...,15.460,0.0,0.0,...,12.080,0.000,0.000,3.200,2.275,2.275,2.275,4.43375,3.58875,4.38875
3,4,KYN DRYL INC,KYN DRYL INC has launched a new initiative to ...,KYN DRYL INC has reported progress towards its...,Reports have surfaced alleging health and safe...,KYN DRYL INC has been actively involved in sup...,KYN DRYL INC has launched a new initiative to ...,20.030,0.0,0.0,...,10.180,0.000,4.595,0.000,0.000,7.065,7.065,5.00750,5.46000,4.31125
4,5,CAPGEMINI US,Capgemini US has introduced an advanced analyt...,Capgemini US has come under fire for allegatio...,The company has been praised for its 'Tech for...,Capgemini US has achieved significant strides ...,Capgemini US has introduced an advanced analyt...,17.015,0.0,0.0,...,0.000,0.000,7.830,0.000,0.000,14.985,14.985,4.25375,5.70375,3.74625


In [49]:
company.to_csv("esg_score_final.csv")